In [186]:
import torch
import torch.nn as nn

from tqdm import tqdm
from torch.optim import SGD
from sklearn.datasets import make_moons
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [187]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.coef = nn.Parameter(torch.randn((1, 2)))
        self.bias   = nn.Parameter(torch.randn(1))
    
    def forward(self, x):
        return 1 / (1 + torch.exp(-(self.bias + torch.sum(self.coef * x, dim = 1, keepdims = True))))

In [188]:
class Dataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx].unsqueeze(dim = -1).to(torch.float64)

In [189]:
X, y = make_moons(n_samples = 10000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)

In [190]:
train_dataset = Dataset(X_train, y_train)
test_dataset  = Dataset(X_test,  y_test)
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_dataloader  = DataLoader(test_dataset,  batch_size = 1, shuffle = False)

In [191]:
model = LogisticRegression()
optimizer = SGD(model.parameters())
criterion = nn.BCELoss()

In [192]:
model.train()

n_epochs = 250
for epoch in range(n_epochs):
    print(f'Epoch {epoch + 1}/{n_epochs}')

    total_loss = 0
    for x, y_true in tqdm(train_dataloader):
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y_true)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    print('Training loss:', total_loss / len(train_dataloader))



Epoch 1/250


100%|██████████| 250/250 [00:00<00:00, 508.34it/s]


Training loss: 0.860366883761883
Epoch 2/250


100%|██████████| 250/250 [00:00<00:00, 534.14it/s]


Training loss: 0.8191184048474529
Epoch 3/250


...

100%|██████████| 250/250 [00:00<00:00, 670.18it/s]


Training loss: 0.26440635829733194
Epoch 218/250


100%|██████████| 250/250 [00:00<00:00, 667.56it/s]


Training loss: 0.2642463228955471
Epoch 219/250


 50%|█████     | 125/250 [00:00<00:00, 623.12it/s]

In [ ]:
model.eval()

with torch.no_grad():
    total_loss = 0
    correct = 0
    for x, y_true in tqdm(test_dataloader):
        y_pred = model(x)
        if torch.round(y_true) == torch.round(y_pred):
            correct += 1
        loss = criterion(y_pred, y_true)
        total_loss += loss.item()
        
    print('Test loss:', total_loss / len(test_dataloader))
    print('Accuracy:', correct / len(test_dataloader)) # TODO: substituir por acurácia balanceada

  0%|          | 0/2000 [00:00<?, ?it/s]

100%|██████████| 2000/2000 [00:01<00:00, 1832.20it/s]

Test loss: 0.23434035190377164
Accuracy: 0.889
